## Exercice 1 : Scraping de données textuelles

### Scraping de données sur le site de Burkina Info

# Installation des bibliothèques nécessaires

In [2]:
# Installation des bibliothèques nécessaires
%pip install requests beautifulsoup4 pandas

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached pandas-2.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 16.8 kB/s eta 0:00:0031m16.4 kB/s eta 0:00:01
Using cached pandas-2.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
Using cache

In [11]:
# Importation des bibliothèques
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import os
from urllib.parse import urljoin

In [12]:
# Liste des rubriques à scraper
rubriques = {
   "politique": "https://burkinainfo.com/category/politique/",
    "economie": "https://burkinainfo.com/category/economie/",
    "societe" : "https://burkinainfo.com/category/societe/",
    "culture" : "https://burkinainfo.com/category/culture/",
    "sport" : "https://burkinainfo.com/category/sports/",
    "tech" : "https://burkinainfo.com/category/tie-tech/"
}

In [13]:
# Fonction de scraping
def scrap_burkinainfo(pages=5):
    articles = []
    
    # Headers pour éviter les blocages
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'fr-FR,fr;q=0.9,en;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    
    for rubrique, url in rubriques.items():
        print(f"Scraping de la rubrique: {rubrique}")
        
        for page in range(1, pages + 1):
            print(f" Page {page}")
            
            # Construction de l'URL pour les pages suivantes
            if page == 1:
                page_url = url
            else:
                page_url = f"{url}page/{page}/"
                
            try:
                response = requests.get(page_url, headers=headers, timeout=10)
                
                # Vérification du succès de la requête
                if response.status_code != 200:
                    print(f" Erreur {response.status_code} pour la page {page} de {rubrique}")
                    continue
                    
                # Parsing du contenu HTML
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # Debug: afficher la structure HTML pour comprendre
                if page == 1 and rubrique == "politique":
                    print("DEBUG: Structure HTML trouvée:")
                    # Chercher différents patterns possibles
                    possible_selectors = [
                        'article',
                        '.post',
                        '.entry',
                        '[class*="post"]',
                        'h1 a, h2 a, h3 a',
                        'a[href*="burkinainfo.com/20"]'  # liens d'articles avec date
                    ]
                    
                    for selector in possible_selectors:
                        elements = soup.select(selector)
                        if elements:
                            print(f"  - Trouvé {len(elements)} éléments avec '{selector}'")
                
                # Méthode 1: Chercher tous les liens qui semblent être des articles
                article_links = soup.find_all('a', href=re.compile(r'/\d{4}/\d{2}/\d{2}/'))
                
                print(f" Trouvé {len(article_links)} liens d'articles potentiels")
                
                for link in article_links:
                    try:
                        href = link.get('href')
                        title_text = link.get_text(strip=True)
                        
                        # Éviter les liens "Lire la suite"
                        if not title_text or 'Lire la suite' in title_text or len(title_text) < 10:
                            continue
                            
                        # Vérifier que c'est un lien complet
                        if href and href.startswith('http'):
                            full_link = href
                        elif href:
                            full_link = urljoin("https://burkinainfo.com", href)
                        else:
                            continue
                            
                        # Éviter les doublons
                        if not any(art['link'] == full_link for art in articles):
                            articles.append({
                                'title': title_text,
                                'link': full_link,
                                'rubrique': rubrique
                            })
                            
                    except Exception as e:
                        print(f"   Erreur lors du traitement d'un lien: {e}")
                        continue
                
                # Méthode 2: Si la méthode 1 ne fonctionne pas, essayer d'autres approches
                if not article_links:
                    print(" Méthode 1 échouée, essai de la méthode 2...")
                    
                    # Chercher dans des containers plus génériques
                    containers = soup.find_all(['div', 'section', 'article'], class_=re.compile(r'post|entry|article|content'))
                    
                    for container in containers:
                        title_elem = container.find(['h1', 'h2', 'h3', 'h4'], recursive=True)
                        if title_elem:
                            link_elem = title_elem.find('a') or container.find('a')
                            if link_elem and link_elem.get('href'):
                                href = link_elem.get('href')
                                title = title_elem.get_text(strip=True)
                                
                                if title and len(title) > 10 and 'Lire la suite' not in title:
                                    full_link = urljoin("https://burkinainfo.com", href) if not href.startswith('http') else href
                                    
                                    if not any(art['link'] == full_link for art in articles):
                                        articles.append({
                                            'title': title,
                                            'link': full_link,
                                            'rubrique': rubrique
                                        })
                
                print(f" Articles extraits de cette page: {len([a for a in articles if a['rubrique'] == rubrique]) - len([a for a in articles[:-(len(article_links) if article_links else 0)] if a['rubrique'] == rubrique])}")
                
                # Pause entre les requêtes pour éviter d'être bloqué
                time.sleep(1)
                
            except requests.RequestException as e:
                print(f" Erreur de connexion pour la page {page} de {rubrique}: {e}")
                continue
            except Exception as e:
                print(f" Erreur inattendue pour la page {page} de {rubrique}: {e}")
                continue
    
    print(f"\nTotal des articles récupérés: {len(articles)}")
    
    if articles:
        # Conversion en DataFrame et sauvegarde
        df = pd.DataFrame(articles)
        
        # Supprimer les doublons basés sur le lien
        df = df.drop_duplicates(subset=['link'], keep='first')
        
        # Créer le dossier s'il n'existe pas
        import os
        os.makedirs('Docs', exist_ok=True)
        
        # Sauvegarde
        df.to_csv('Docs/burkinainfo_articles.csv', index=False, encoding='utf-8-sig')
        print(f"Fichier sauvegardé avec {len(df)} articles uniques")
        
        return df
    else:
        print("Aucun article trouvé!")
        # Retourner un DataFrame vide avec les bonnes colonnes
        return pd.DataFrame(columns=['title', 'link', 'rubrique'])



In [14]:
# Test de la fonction
if __name__ == "__main__":
    df_articles = scrap_burkinainfo(pages=2)  # Commencer avec 2 pages pour tester
    print(f"Total des articles scrapés: {len(df_articles)}")
    if not df_articles.empty:
        print("\nPremiers articles:")
        print(df_articles.head(10))
    else:
        print("Le DataFrame est vide!")

Scraping de la rubrique: politique
 Page 1
DEBUG: Structure HTML trouvée:
  - Trouvé 10 éléments avec '.post'
  - Trouvé 103 éléments avec '[class*="post"]'
  - Trouvé 10 éléments avec 'h1 a, h2 a, h3 a'
  - Trouvé 108 éléments avec 'a[href*="burkinainfo.com/20"]'
 Trouvé 46 liens d'articles potentiels
 Articles extraits de cette page: 18
 Page 2
 Trouvé 46 liens d'articles potentiels
 Articles extraits de cette page: 28
Scraping de la rubrique: economie
 Page 1
 Trouvé 58 liens d'articles potentiels
 Articles extraits de cette page: 10
 Page 2
 Trouvé 61 liens d'articles potentiels
 Articles extraits de cette page: 20
Scraping de la rubrique: societe
 Page 1
 Trouvé 66 liens d'articles potentiels
 Articles extraits de cette page: 8
 Page 2
 Trouvé 66 liens d'articles potentiels
 Articles extraits de cette page: 18
Scraping de la rubrique: culture
 Page 1
 Trouvé 46 liens d'articles potentiels
 Articles extraits de cette page: 10
 Page 2
 Trouvé 42 liens d'articles potentiels
 Articles